In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    
    combined_df.to_csv(f"url_data.csv", index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

    return len(failed_downloads) == 0  # 모든 다운로드가 성공했으면 True 반환



def compress_images_to_zip(download_path, post_href, post_user):
#     print(post_href)
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    duplicate_url = combine_csv_files_remove_duplicates('.')
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urls = duplicate_url.loc[:, 'url'].to_list()
    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href in old_urls : 
                print(f"Processing: {total} 중 {i}  {title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href, title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [2]:

download_path = '/Users/jonathanoh/Desktop/down'
excel_path = '/Users/jonathanoh/Desktop/down/image_url.xlsx'

options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [3]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

[('https://arca.live/b/aiartreal/95029851?p=1', '사이키델릭 아트', '빠다파리'),
 ('https://arca.live/b/aiartreal/95029688?p=1', '이집트 공주', '빠다파리'),
 ('https://arca.live/b/aiartreal/95029446?p=1', '귀여운 서큐버스', '빠다파리'),
 ('https://arca.live/b/aiartreal/95027173?p=1',
  '배경화면 바꾸려고 몇장뽑아봤어요...',
  '크런키z'),
 ('https://arca.live/b/aiartreal/95026696?p=1', '캐주얼 드레스', '바코드'),
 ('https://arca.live/b/aiartreal/95025167?p=1',
  'AI와 체팅으로 건진 약후방짤등. part4',
  '빛바랜쏠로'),
 ('https://arca.live/b/aiartreal/95024790?p=1',
  '와 이거 너무 아쉬운데',
  'Invalid_Syntax'),
 ('https://arca.live/b/aiartreal/95023418?p=1',
  'cafe - 바코드 프롬 돚거',
  '머리는차갑게가슴은뜨겁게'),
 ('https://arca.live/b/aiartreal/95023245?p=1', '공유된 와카로 뽑아봄', 'yong91892'),
 ('https://arca.live/b/aiartreal/95021947?p=1',
  '[스압] 판타지 복장과 군복',
  'qwenty230'),
 ('https://arca.live/b/aiartreal/95020730?p=1', '붉은 악마', '산체스'),
 ('https://arca.live/b/aiartreal/95019728?p=1',
  '하니믹스 같은 모델들 반으로 줄여도 괜찮죠?',
  'kiki11'),
 ('https://arca.live/b/aiartreal/95019527?p=1',
  '고스로리 (go

In [36]:
post_list = []
for i in range(3):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)
post_list    

[('https://arca.live/b/aiartreal/95042010?p=1',
  '병합놀이 하면서 2.7D Swimsuit 테스트 27장 / Cindy K9 향첨가',
  '왼손메로나'),
 ('https://arca.live/b/aiartreal/95042001?p=1', '즐겁게 웃는 여자', '냥이집사'),
 ('https://arca.live/b/aiartreal/95040549?p=1', '1girls', 'aborim'),
 ('https://arca.live/b/aiartreal/95039151?p=1', '((blush:1.3)) 차이', '이노프리'),
 ('https://arca.live/b/aiartreal/95038304?p=1', 'leather light armor', '바코드'),
 ('https://arca.live/b/aiartreal/95038298?p=1', '얻으려면...', 'MR_OWL'),
 ('https://arca.live/b/aiartreal/95037817?p=1',
  'hanimix_real_v15B 부먹',
  '빠다파리'),
 ('https://arca.live/b/aiartreal/95036470?p=1',
  '보석 (hanimix_real_v15B)',
  '너예쁘다'),
 ('https://arca.live/b/aiartreal/95036112?p=1', '에반게리온', '빠다파리'),
 ('https://arca.live/b/aiartreal/95035941?p=1', '동굴', '빠다파리'),
 ('https://arca.live/b/aiartreal/95035774?p=1', '징크스 (LOL)', '빠다파리'),
 ('https://arca.live/b/aiartreal/95035594?p=1',
  'hanimix_real_v15A 부먹',
  '빠다파리'),
 ('https://arca.live/b/aiartreal/95034639?p=1',
  '앞면 뒷면 동시에 나오는 방법이

In [44]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 병합놀이 하면서 2.7D Swimsuit 테스트 27장 / Cindy K9 향첨가
중복 : 즐겁게 웃는 여자
중복 : 1girls
중복 : ((blush:1.3)) 차이
중복 : leather light armor
중복 : 얻으려면...
중복 : hanimix_real_v15B 부먹
중복 : 보석 (hanimix_real_v15B)
중복 : 에반게리온
중복 : 동굴
중복 : 징크스 (LOL)
중복 : hanimix_real_v15A 부먹
중복 : 앞면 뒷면 동시에 나오는 방법이 뭔가요?
중복 : 건전한 청청패션
중복 : NeMix F v2.8.0 찍먹! 서술형 프롬을 잘 알아듣는 모델!
중복 : AI 이미지 + 소설로 수익화 해보고 싶으신 분은 없나요?
중복 : 사이키델릭 아트
중복 : 이집트 공주
중복 : 귀여운 서큐버스
중복 : 배경화면 바꾸려고 몇장뽑아봤어요...
중복 : 캐주얼 드레스
중복 : AI와 체팅으로 건진 약후방짤등. part4
중복 : 와 이거 너무 아쉬운데
중복 : cafe - 바코드 프롬 돚거
중복 : 공유된 와카로 뽑아봄
중복 : [스압] 판타지 복장과 군복
중복 : 붉은 악마
중복 : 하니믹스 같은 모델들 반으로 줄여도 괜찮죠?
중복 : 고스로리 (gothic lolita) 23장
중복 : Open Clothes,
중복 : 처음엔 빈티지 컨셉이었는데...
중복 : thong - 픽시브 정지하게 만든 그림
중복 : 마술사 (hanimix_real_v15A)
중복 : hanimix_real_v15A 사용 - 얼굴위주 (모델은 고퀄 사용자는 저퀄)
중복 : Pool & Swimsuit
중복 : 생초보^^:: 머리는차갑게가슴은뜨겁게님 ~~ 와카 로 맹글어 보아요
중복 : 벚꽃 란제리 기모노 영상
중복 : 니트 & 펜슬 스커트
중복 : NeMix F v2.8.0 + 리스펙트님 와카 + LCM 조합 찍먹
중복 : 제가 사용하는 ComfyUI 워크플로우 수정 버전...
중복 : 도시를 거니는 고양이녀
중복 : 얼굴 클로즈업
중복 : 화이트

In [30]:
list = process_images(driver, list, download_path)
# list = process_images(driver, re, download_path)

Processing: 9 중 1  leather light armor  _  바코드
https://arca.live/b/aiartreal/95038304?p=1
Processing: 9 중 2  보석 (hanimix_real_v15B)  _  너예쁘다
https://arca.live/b/aiartreal/95036470?p=1
Processing: 9 중 3  sdxl confyui 속도  _  출산율
https://arca.live/b/aiartreal/94999170?p=2
failed : https://arca.live/b/aiartreal/94999170?p=2
Processing: 9 중 4  에너지 흐름  _  빠다파리
https://arca.live/b/aiartreal/94995361?p=2
Processing: 9 중 5  hanimix_신버젼(4개 모델) 공유합니다  _  하니타카
https://arca.live/b/aiartreal/94992489?p=2
failed : https://arca.live/b/aiartreal/94992489?p=2
Processing: 9 중 6  LCM 테스트  _  adoniel
https://arca.live/b/aiartreal/94970803?p=3
failed : https://arca.live/b/aiartreal/94970803?p=3
Processing: 9 중 7  익스텐션텝 무한로딩 도와주세요  _  mone97
https://arca.live/b/aiartreal/94967781?p=3
failed : https://arca.live/b/aiartreal/94967781?p=3
Processing: 9 중 8  달빛 아래 레깅스 여자  _  리버스Q
https://arca.live/b/aiartreal/94966780?p=3
failed : https://arca.live/b/aiartreal/94966780?p=3
Processing: 9 중 9  NeMix VurfY / 제한 공유 /

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [41]:
temp = []
for url, title, user in list :
    temp.append([url, title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
df.to_csv("fail_list.csv", index=False)
df = combine_csv_files_remove_duplicates('.')